In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import ccxt

In [41]:
# Inicializar o objeto da exchange (Binance neste exemplo)
exchange = ccxt.binance()

# Definir o par de negociação e o intervalo de tempo
symbol = ["ETH/USDT","BTC/USDT"]  # ou 'BTC/USD' dependendo da exchange
timeframe = '1d'  # Pode ser '1m', '5m', '1h', '1d', etc.

# Funções Auxiliares

### Definindo o algoritmo para simular as operações

In [42]:
import math

# Create a function to round any number to the smalles multiple of 100
def round_down(x):
    return int(math.floor(x / 100.0)) * 100

def backtest_algorithm(
    df,
    capital_exposure,
    initial_capital):

    # List with the total capital after every operation
    total_capital = [initial_capital]

    # List with profits for every operation
    all_profits = [] 

    ongoing = False

    for i in range(0,len(df)):

        if ongoing == True:

            if (df["Open"][i] >= target) | (df["Open"][i] <= stop): 
                exit = df["Open"][i]

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["Low"][i] <= stop: 
                exit = stop

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["High"][i] >= target: 
                exit = target

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

        else:
            if ~(np.isnan(df["buy_price"][i])):
                entry = df["buy_price"][i]
                stop = df["stop"][i]
                
                if df["Low"][i] > stop: 
                    ongoing = True
                    risk = entry - stop
                    target = df["target"][i]
                    shares = round_down(capital_exposure / risk)

    return all_profits, total_capital

### Calculando a estatística e a curva de capital

In [43]:
def get_drawdown(data, column = "Close"):
    data["Max"] = data[column].cummax()
    data["Delta"] = data['Max'] - data[column]
    data["Drawdown"] = 100 * (data["Delta"] / data["Max"])
    max_drawdown = data["Drawdown"].max()
    return max_drawdown

def strategy_test(all_profits, total_capital):
    if len(all_profits) > 0:
        gains = sum(x >= 0 for x in all_profits)
        losses = sum(x < 0 for x in all_profits)
        num_operations = gains + losses
        pct_gains = 100 * (gains / num_operations)
        pct_losses = 100 - pct_gains
        total_profit = sum(all_profits)
        pct_profit = (total_profit / total_capital[0]) * 100
        
        # Compute drawdown
        total_capital = pd.DataFrame(data=total_capital, columns=["total_capital"])
        drawdown = get_drawdown(data=total_capital, column="total_capital")

        # Compute profit per operation
        profit_per_operation = pct_profit / num_operations

        # Expeceted Value
        all_positives = [x for x in all_profits if x >= 0]
        if len(all_positives) > 0:
            average_gain = sum(all_positives) / len(all_positives)
        else:
            average_gain = 0

        all_negatives = [x for x in all_profits if x < 0]
        if len(all_negatives) > 0:
            average_loss = sum(all_negatives) / len(all_negatives)
        else:
            average_loss = 0
            
        num_operations = len(all_profits)
        pct_gains = (len(all_positives) / num_operations)
        pct_losses = 1 - pct_gains
        
        expected_value = (average_gain * pct_gains) + (average_loss * pct_losses)
    else:
        total_profit = 0
        pct_profit = 0
        drawdown = 0
        num_operations = 0
        profit_per_operation = 0
        gains = 0
        pct_gains = 0
        losses = 0
        pct_losses = 0
        expected_value = 0

    return {
        "total_profit": total_profit,
        "pct_profit": pct_profit,
        "pct_drawdown": drawdown,
        "num_operations": num_operations,
        "pct_profit_per_operation": profit_per_operation,
        "num_gains": gains ,
        "pct_gains": pct_gains,
        "num_losses": losses,
        "pct_losses": pct_losses, 
        "expected_value": expected_value
    }
def capital_plot(total_capital, all_profits):
  all_profits = [0] + all_profits # make sure both lists are the same size
  cap_evolution = pd.DataFrame({'Capital': total_capital, 'Profit': all_profits})
  plt.title("Curva de Capital")
  plt.xlabel("Total Operações")
  cap_evolution['Capital'].plot()

# Ajuste do DF para chamada das Funções de modo Escalável

In [44]:
df_stats = pd.DataFrame()
for stock in symbol:
    # Obter dados OHLC
    ohlcv = exchange.fetch_ohlcv(stock, timeframe)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp',inplace=True)

### Definindo o candle sinal
    condition_1 = df["Low"] > df["Low"].shift(1)
    condition_2 = df["Low"].shift(1) < df["Low"].shift(2)
    df["signal"] = condition_1 & condition_2

### Calculando o Éden dos Traders
    df["mme8"] = df["Close"].ewm(span=8, min_periods=8).mean()
    df["mme80"] = df["Close"].ewm(span=80, min_periods=80).mean()
    df["eden"] = (df["mme8"] > df["mme8"].shift(1)) & (df["mme80"] > df["mme80"].shift(1)) #& (df["mme8"] > df["mme80"])
    df.dropna(inplace=True)

### Definindo o preço de compra
    condition_1 = df["signal"].shift(1) == True
    condition_2 = df["eden"].shift(1) == True 
    condition_3 = df["High"] > df["High"].shift(1)
    tick = 0.01

    df["buy_price"] = np.where(
        condition_1 & condition_2 & condition_3, 
        np.where(df["Open"] > df["High"].shift(1), df["Open"], df["High"].shift(1) + tick),
        np.nan)
    
### Definindo o alvo
    max_high = df["High"].rolling(3).max()
    min_low = df["Low"].rolling(3).min()
    amplitude = (max_high.shift(1) - min_low.shift(1))
    entry = df["High"].shift(1)
    df["target"] =  amplitude + entry

### Definindo o stop
    df["stop"] = df["Low"].shift(2) - tick

### Realizando o Backtest
    all_profits, total_capital = backtest_algorithm(
    df=df,
    capital_exposure=100000,
    initial_capital=100000)
    statistics = strategy_test(all_profits, total_capital)
    statistics = pd.DataFrame.from_dict(statistics, orient='index').round(2)
    df_stats[stock] = statistics
df_stats = df_stats.T

In [45]:
df_stats.sort_values('total_profit').tail(20)

,total_profit,pct_profit,pct_drawdown,num_operations,pct_profit_per_operation,num_gains,pct_gains,num_losses,pct_losses,expected_value
BTC/USDT,102869.0,102.87,41.96,18.0,5.71,15.0,0.83,3.0,0.17,5714.94
ETH/USDT,572956.0,572.96,73.19,20.0,28.65,11.0,0.55,9.0,0.45,28647.80
